In [1]:
# Essential imports:
import pandas as pd
import lucene
import os
import re
from java.io import File

# Indexer imports:
from org.apache.lucene.analysis.standard import StandardAnalyzer
from org.apache.lucene.store import FSDirectory
import org.apache.lucene.document as document

# Retriever imports:
from org.apache.lucene.index import IndexReader
from org.apache.lucene.search import IndexSearcher
from org.apache.lucene.index import DirectoryReader
from org.apache.lucene.queryparser.classic import QueryParser

from java.io import File
lucene.initVM()

In [2]:
# class CustomTFIDFSimilarity(ClassicSimilarity):
    
#     print("Welcome")
#     # query_length = 0
#     # def __init__(self, qlength):
#     #     self.query_length = qlength

#     def tf(freq):
#         print("TF called")
#         unique_terms = FieldInvertState.getUniqueTermCount() 
#         doc_length = FieldInvertState.getLength()
#         AvgTF = doc_length / unique_terms
#         RITF = math.log2(1+freq) / math.log2(1+AvgTF)
#         total_freq = CollectionStatistics.sumTotalTermFreq()
#         num_docs = CollectionStatistics.maxDoc()
#         ADL = total_freq / num_docs
#         LRTF = freq * math.log2(1+(ADL/doc_length))
#         BRITF = RITF / (1+RITF)
#         BLRTF = LRTF / (1+LRTF)
#         w = 2 / (1 + math.log2(a+query_length))
#         TFF = ( w * BRITF ) + ( (1-w) * BLRTF )
#         return TFF

#     def idf(docFreq, docCount):
        
#         raw_idf = math.log((docCount + 1) / docFreq )
#         CTF = TermStatistics.totalTermFreq()
#         AEF = CTF / freq
#         TDF = raw_idf * (AEF/(1+AEF))
#         return TDF
    
#     def lengthNorm(numTerms):
#         # Custom length normalization
#         return 1.0
#     def TFIDFScorer(boost, idf, normTable):
#         self.idf = idf
#         self.boost = 1
#         self.queryWeight = idf.getValue().floatValue()
#         self.normTable = 1
#     def score(freq, norm):
#         raw = tf(freq)*queryWeight
#         return 2*raw

In [3]:
indexPath = File("index/").toPath()
indexPath

<Path: index>

In [4]:
from java.nio.file import Paths
from org.apache.lucene.index import DirectoryReader, Term

index_directory = FSDirectory.open(Paths.get("index/"))
reader = DirectoryReader.open(index_directory)

# 1. Term Frequency (term, doc)
term = Term('TEXT',"example")
doc_id = 0  # Example document ID
field = "TEXT"

docfreq = reader.docFreq(term)
print(docfreq)
numDoc = reader.numDocs()
print(numDoc)
num_terms = 0
for i in range(numDoc):
    term_vector = reader.getTermVector(i,field)
    num_terms += term_vector.getSumTotalTermFreq()
ADL = num_terms / numDoc
print(ADL)

38352
527979
504.6469423973302


In [5]:
import math
from org.apache.lucene.analysis.standard import StandardAnalyzer
from org.apache.lucene.search import IndexSearcher
from org.apache.lucene.util import BytesRef, BytesRefIterator
from org.apache.lucene.index import DirectoryReader, Term, TermsEnum
analyzer = StandardAnalyzer()
directory = FSDirectory.open(indexPath)
searcher = IndexSearcher(DirectoryReader.open(directory))

def scored_docs(query,numDoc):

    terms = query.split()
    query_length = len(terms)
    w = 2 / (1 + math.log2(1 + query_length))
    
    field = "TEXT"
        
    document_dict = {}
    query = QueryParser(field, analyzer).parse(query)
    
    scoreDocs = searcher.search(query, numDoc).scoreDocs
    
    for i in range(len(scoreDocs)):

        doc_id = scoreDocs[i].doc
        doc = searcher.doc(scoreDocs[i].doc)
        
        term_vector = reader.getTermVector(doc_id,field)
        termsEnumvar = term_vector.iterator()
        termsref = BytesRefIterator.cast_(termsEnumvar)
        term_dict = {}
        
        while (termsref.next()):
            
            termval = TermsEnum.cast_(termsref)
            term_doc = termval.term().utf8ToString()
            tf_doc = termsEnumvar.totalTermFreq() 
            term_dict[term_doc] = tf_doc
            
        # print(term_dict)   
        len_D = sum(term_dict.values())
        num_terms = len(term_dict)
        average_tf = len_D / num_terms

        sum_TDF = 0
        sim = 0

        for term in terms :
            if term in term_dict:
                tf = term_dict[term]
            else:
                tf = 0
            RITF = math.log2(1+tf) / math.log2(1+average_tf) 
            LRTF = tf * math.log2(1 + (ADL/len_D))
            BRITF = RITF / (1 + RITF)
            BLRTF = LRTF / (1 + LRTF)
            
            TFF = (w * BRITF) + ((1 - w) * BLRTF)
        
            docfreq = reader.docFreq(Term('TEXT',term))
            numDoc = reader.numDocs()

            if docfreq != 0:
                IDF = math.log2((numDoc+1)/docfreq)
                CTF = reader.totalTermFreq(Term("TEXT", term))
                AEF =  CTF/ docfreq
                TDF = IDF * (AEF / (1 + AEF))
            else :
                TDF = 0
            sum_TDF += TDF
            sim += TFF * TDF
    
        document_dict[doc.get("DOC_NO")] = sim / sum_TDF
        term_dict.clear()
    return document_dict


In [6]:
def write_file(document_dict,query_no):
    if query_no <= 450 :
        with open("trec678.txt", "a") as file:
            i = 1
            for key, value in document_dict.items():
                file.write("%d\t Q0\t %s\t %d\t %f\t CS2320\n"%(query_no, key, i, value))
                i += 1
                if (i>1000):
                    break
    else :
        with open("robust.txt", "a") as file:
            i = 1
            for key, value in document_dict.items():
                file.write("%d\t Q0\t %s\t %d\t %f\t CS2320\n"%(query_no, key, i, value))
                i += 1
                if (i>1000):
                    break

In [14]:
import os
import re
stop_words = {'a','about','above','after','again','against','ain','all','am','an','and','any','are','aren',"aren't",'as','at','be','because','been','before','being','below','between','both','but','by','can','couldn',"couldn't",'d','did','didn',"didn't",'do','does','doesn',"doesn't",'doing','don',"don't",'down','during','each','few','for','from','further','had','hadn',"hadn't",'has','hasn',"hasn't",'have','haven',"haven't",'having','he','her','here','hers','herself','him','himself','his','how','i','if','in','into','is','isn',"isn't",'it',"it's",'its','itself','just','ll','m','ma','me','mightn',"mightn't",'more','most','mustn',"mustn't",'my','myself','needn',"needn't",'no','nor','not','now','o','of','off','on','once','only','or','other','our','ours','ourselves','out','over','own','re','s','same','shan',"shan't",'she',"she's",'should',"should've",'shouldn',"shouldn't",'so','some','such','t','than','that',"that'll",'the','their','theirs','them','themselves','then','there','these','they','this','those','through','to','too','under','until','up','ve','very','was','wasn',"wasn't",'we','were','weren',"weren't",'what','when','where','which','while','who','whom','why','will','with','won',"won't",'wouldn',"wouldn't",'y','you',"you'd","you'll","you're","you've",'your','yours','yourself','yourselves'}
def query_parser(file_path):

    i = 1
    list_query = []
    
    directory = os.fsencode(file_path)
    
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        f = open(file_path+filename,encoding="latin")
        line = f.readline()
        
        print("Processing file %dth file -"%i,filename)
        i += 1
        while line :
            if "<num>" in line :
                num = re.sub('<[/]*\w+>', '', line).strip()
            if "<title>" in line :
                query = re.sub('<[/]*\w+>', '', line).lower().strip()
                input_string = query.lower()
                special_chars_pattern = re.compile(r'[^a-zA-Z0-9\s]')
                query = re.sub(special_chars_pattern, ' ', input_string).split()
                filtered_sentence = [word for word in query if word.lower() not in stop_words]
                query = ' '.join(filtered_sentence)
                list_query.append((num,query))
            line = f.readline()
    return list_query

list_query = query_parser("trec678rb/topics/")
len(list_query)

Processing file 1th file - trec678.xml
Processing file 2th file - robust.xml


250

In [15]:
list_query

[('301', 'international organized crime'),
 ('302', 'poliomyelitis post polio'),
 ('303', 'hubble telescope achievements'),
 ('304', 'endangered species mammals'),
 ('305', 'dangerous vehicles'),
 ('306', 'african civilian deaths'),
 ('307', 'new hydroelectric projects'),
 ('308', 'implant dentistry'),
 ('309', 'rap crime'),
 ('310', 'radio waves brain cancer'),
 ('311', 'industrial espionage'),
 ('312', 'hydroponics'),
 ('313', 'magnetic levitation maglev'),
 ('314', 'marine vegetation'),
 ('315', 'unexplained highway accidents'),
 ('316', 'polygamy polyandry polygyny'),
 ('317', 'unsolicited faxes'),
 ('318', 'best retirement country'),
 ('319', 'new fuel sources'),
 ('320', 'undersea fiber optic cable'),
 ('321', 'women parliaments'),
 ('322', 'international art crime'),
 ('323', 'literary journalistic plagiarism'),
 ('324', 'argentine british relations'),
 ('325', 'cult lifestyles'),
 ('326', 'ferry sinkings'),
 ('327', 'modern slavery'),
 ('328', 'pope beatifications'),
 ('329', '

In [16]:
for query in list_query:
    document_dict = scored_docs(query[1],numDoc)
    sorted_document_dict = dict(sorted(document_dict.items(), key=lambda item: item[1],reverse=True))
    write_file(sorted_document_dict,int(query[0]))
    print("Processes query %d"%int(query[0]))

Processes query 301
Processes query 302
Processes query 303
Processes query 304
Processes query 305
Processes query 306
Processes query 307
Processes query 308
Processes query 309
Processes query 310
Processes query 311
Processes query 312
Processes query 313
Processes query 314
Processes query 315
Processes query 316
Processes query 317
Processes query 318
Processes query 319
Processes query 320
Processes query 321
Processes query 322
Processes query 323
Processes query 324
Processes query 325
Processes query 326
Processes query 327
Processes query 328
Processes query 329
Processes query 330
Processes query 331
Processes query 332
Processes query 333
Processes query 334
Processes query 335
Processes query 336
Processes query 337
Processes query 338
Processes query 339
Processes query 340
Processes query 341
Processes query 342
Processes query 343
Processes query 344
Processes query 345
Processes query 346
Processes query 347
Processes query 348
Processes query 349
Processes query 350
